In [1]:
import pinocchio
import numpy as np 
import time 
import pymanopt 
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem

from utils import *
import networkx as nx
from scipy import optimize
from scipy.optimize import Bounds

import tqdm

LOWER = "lower_limit"
UPPER = "upper_limit"
BOUNDED = "bounded"
BELOW = "below"
ABOVE = "above"
TYPE = "type"
OBSTACLE = "obstacle"
ROBOT = "robot"
END_EFFECTOR = "end_effector"
RADIUS = "radius"
DIST = "weight"
POS = "pos"
BASE = "base"
ANCHOR = "anchor"
UNDEFINED = None

deg2rad = np.pi/180.
rad2deg = 180./np.pi

scale = 1

%load_ext autoreload
%autoreload 2

In [2]:
def create_base_graph(model,data,axis_length):

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))

    q_init = pinocchio.neutral(model)
    pinocchio.forwardKinematics(model,data,q_init)
    
    robot_name = model.names[1]
    ROOT = robot_name

    base = nx.empty_graph()

    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            cur_pos, aux_cur_pos = (
                oMi.translation*scale,
                pinocchio.SE3.act(pinocchio.SE3(oMi.rotation,oMi.translation*scale),trans_z).translation,
            )
            #print(name,cur_pos,aux_cur_pos)
            dist = np.linalg.norm(cur_pos - aux_cur_pos)

            # Add nodes for joint and edge between them
            base.add_nodes_from(
                [(cur, {POS: cur_pos}), (aux_cur, {POS: aux_cur_pos})]
            )
            base.add_edge(
                cur, aux_cur, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False}
            )

            # If there exists a preceeding joint, connect it to new
            if idx != 0:
                pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')
                for u in [pred, aux_pred]:
                    for v in [cur, aux_cur]:
                        dist = np.linalg.norm(
                            base.nodes[u][POS] - base.nodes[v][POS]
                        )
                        base.add_edge(
                            u,
                            v,
                            **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False},
                        )
        elif idx == 1:
            #print(name,oMi.translation)
            base = nx.DiGraph(
                [
                    (robot_name, "x"),
                    (robot_name, "y"),
                    (robot_name, robot_name+'_tilde'),
                    ("x", "y"),
                    ("y", robot_name+'_tilde'),
                    (robot_name+'_tilde' ,"x"),
                ]
            )
            base.add_nodes_from(
                [
                    ("x", {POS: np.array([axis_length, 0, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    ("y", {POS: np.array([0, -axis_length, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    (robot_name, {POS: oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                    (robot_name+'_tilde', {POS: np.array([0, 0, axis_length]) + oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                ]
            )
            for u, v in base.edges():
                base[u][v][DIST] = np.linalg.norm(base.nodes[u][POS] - base.nodes[v][POS])
                base[u][v][LOWER] = base[u][v][DIST]
                base[u][v][UPPER] = base[u][v][DIST]
                base[u][v][ANCHOR] = True
                base[u][v][BOUNDED] = []

    # Set node type to robot
    nx.set_node_attributes(base, [ROBOT], TYPE)
    base.nodes[ROOT][TYPE] = [ROBOT, BASE]
    base.nodes[ROOT + '_tilde'][TYPE] = [ROBOT, BASE]

    return base

In [3]:
axis_length = 0.2*scale
urdf_filename="GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
# 'GraphIK/graphik/robots/urdfs/panda_arm.urdf'

model    = pinocchio.buildModelFromUrdf(urdf_filename)
data     = model.createData()

G = create_base_graph(model,data,axis_length)


In [4]:
nx.get_node_attributes(G,POS)

{'lbr_iiwa_joint_1': array([0.    , 0.    , 0.1575]),
 'x': array([0.2   , 0.    , 0.1575]),
 'y': array([ 0.    , -0.2   ,  0.1575]),
 'lbr_iiwa_joint_1_tilde': array([0.    , 0.    , 0.3575]),
 'lbr_iiwa_joint_2': array([0.  , 0.  , 0.36]),
 'lbr_iiwa_joint_2_tilde': array([-4.13646214e-14,  2.00000000e-01,  3.60000000e-01]),
 'lbr_iiwa_joint_3': array([ 2.07103964e-25, -1.00133990e-12,  5.64500000e-01]),
 'lbr_iiwa_joint_3_tilde': array([ 4.13646214e-14, -1.00131770e-12,  7.64500000e-01]),
 'lbr_iiwa_joint_4': array([ 4.45703796e-14, -1.00131597e-12,  7.80000000e-01]),
 'lbr_iiwa_joint_4_tilde': array([ 8.5935001e-14, -2.0000000e-01,  7.8000000e-01]),
 'lbr_iiwa_joint_5': array([ 8.27292428e-14, -9.78656600e-14,  9.64500000e-01]),
 'lbr_iiwa_joint_5_tilde': array([1.24093864e-13, 1.86078980e-12, 1.16450000e+00]),
 'lbr_iiwa_joint_6': array([1.27299622e-13, 2.01258560e-12, 1.18000000e+00]),
 'lbr_iiwa_joint_6_tilde': array([4.45703796e-14, 2.00000000e-01, 1.18000000e+00]),
 'lbr_iiwa

In [5]:

def norm(x):
    return x/max(np.linalg.norm(x),10**-9)

def goal_graph(model, data, G, axis_length, position, direction = None, anchor = True):
    
    end_effector = [model.names[-1]]
    if direction is not None:
        end_effector.append(model.names[-1]+'_tilde')

    list_nodes_base = ['x','y',model.names[1],model.names[1]+'_tilde']

    for i, cur_end_effector in enumerate(end_effector):
        pos_end = position

        if direction is not None and i == 1:
            pos_end = pos_end + direction*axis_length
        
        G.nodes[cur_end_effector][POS] = pos_end

        for cur_base in list_nodes_base:
            
            if not((i == 1) and (len(end_effector)==1)):
                dist = np.linalg.norm(pos_end-G.nodes[cur_base][POS])

                G.add_edge(
                            cur_base, cur_end_effector, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: anchor}
                        )
    
    if anchor:

        list_edges_anchor = []
        list_other_edges = []
        new_edge_order  = []
        A_anchor = []

        for key ,anch_ in nx.get_edge_attributes(G,ANCHOR).items():

            if anch_:
                list_edges_anchor.append(key)
                A_anchor.append(norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
                #print(key,' : ',norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
            else:
                list_other_edges.append(key)
        new_edge_order = list_edges_anchor.copy() #.extend(list_other_edges)

        for e in list_other_edges:
            new_edge_order.append(e)

        A_anchor = np.array(A_anchor).transpose()
        
    else:

        list_edges_anchor = []
        list_other_edges = [e for e in G.edges()]

        new_edge_order = list_other_edges
        A_anchor = []

    n = len(list_other_edges)
    n_anchor = len(list_edges_anchor)

    weight = nx.get_edge_attributes(G, DIST)
    D = np.diag([weight[i] for i in new_edge_order])

    C = incidence_matrix_(G,oriented=True, edgelist=new_edge_order).toarray()
    C = np.array(C)

    Y_init = np.zeros((3,n))

    for i, e in enumerate(list_other_edges):
        i = i 
        p1 = G.nodes[e[0]][POS]
        p2 = G.nodes[e[1]][POS]
        Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

    if anchor:
        Y_init = [A_anchor,Y_init]
    return G, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init

In [111]:
def simple_IK(d,C,D,A_,n, n_anchor,max_iter,ind=None,W=None,Y_init=None,use_rand=False):
    if  W is None:
        W = np.eye(D.shape[0])
    if n_anchor > 0:    
        if d > 3:
            A = np.zeros((d,n_anchor))
            A[:3,:] = A_[:3,:]
        else:
            if A_.shape[0] == d:
                A = A_
            elif A_.shape[0] > d:
                A = A_[:d,:]
            else:
                A = np.zeros((d,n_anchor))
                A[:3,:] = A_[:3,:]

        manifold_ = manifolds.Product([manifolds.ConstantFactory(A) , manifolds.Oblique(d,n)])
    else:
        manifold_ = manifolds.Oblique(d,n)

    Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D
    Q1 = D@W@D

    if n_anchor > 0:
        zero = np.zeros_like(A)
        @pymanopt.function.numpy(manifold_)
        def cost(A,Y_):
            #print(A.shape,Y_.shape)
            Y = np.concatenate([A,Y_],axis=1)
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(A,Y_):
            Y = np.concatenate([A,Y_],axis=1)
            grad = 2*Q2[n_anchor:,:]@Y.transpose()
            return [zero,grad.transpose() ]
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(A,Y_,U_A, U):
            U_ = np.concatenate([U_A,U],axis=1)
            hess = 2*Q2[n_anchor:,:]@U_.transpose()
            return [zero,hess.transpose()]
    else:
        @pymanopt.function.numpy(manifold_)
        def cost(Y):
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(Y):
            grad = 2*Q2@Y.transpose()
            return grad.transpose() 
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(Y,U):
            hess = 2*Q2@U.transpose()
            return hess.transpose()


    problem = Problem(manifold=manifold_, 
                      cost=cost,
                      euclidean_gradient=euclidean_gradient, 
                      euclidean_hessian=euclidean_hessian,
                      )

    optimizer = optimizers.TrustRegions(max_iterations=max_iter,use_rand=use_rand,min_gradient_norm=10**-12,verbosity=0)
    #print(n_anchor)
    if  Y_init is None:
        Y_init_ = manifold_.random_point()
    elif n_anchor > 1:
        if Y_init[0].shape[0] == d:
            #print(1)
            Y_init_ = Y_init
        elif Y_init[0].shape[0] < d:
            #print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init[0].shape[0],:] = Y_init[1]
            Y_init_ = [A, Y_init_]
        else:
            #print(3)
            if len(Y_init) == 2:
                U,S,V = np.linalg.svd(Y_init[1],full_matrices=False)
            else:
                U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = [A, U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)]
    else:
        if Y_init.shape[0] == d:
            #print(1)
            Y_init_ = Y_init
        elif Y_init.shape[0] < d:
            #print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init.shape[0],:] = Y_init
        else:
            #print(3)
            U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)
    
    Y_star = optimizer.run(problem,initial_point=Y_init_).point

    return Y_star

In [16]:
def direction_to_position(Y,C,D,data):
    #print(Y.shape, C.shape, D.shape)
    X = np.linalg.pinv(C@C.transpose())@C@D@Y.transpose()
    #print(X)
    X = X - X[0,:] + data.oMi[1].translation*scale
    
    #if X[0,0]<0:
      #  X = -X
    
    v = X[1,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    v = X[2,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/np.linalg.norm(e2))
    X = (X - X[0,:])@R.transpose() + X[0,:]
    flip = 1

    if X[3,2]<axis_length:
        flip = -1
    v = X[2,:] - X[0,:]
    R = np.diag((1,1,flip))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    return X

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [91]:
def skew(x):
    """
    Creates a skew symmetric matrix from vector x
    """
    X = np.array([[0, -x[2], x[1]], [x[2], 0, -x[0]], [-x[1], x[0], 0]])
    return X


def joint_variables(model,data, G, T_final, axis_length):
    """
    Finds the set of decision variables corresponding to the
    graph realization G.

    :param G: networkx.DiGraph with known vertex positions
    :param T_final: poses of end-effectors in case two final frames aligned along z
    :returns: Dictionary of joint angles
    :rtype: Dict[str, float]
    """
    tol = 1e-10
    ROOT = model.names[1]
    T = {}
    T[ROOT] = data.oMi[1] #G.nodes[model.names[1]][POS]

    pinocchio.forwardKinematics(model,data,pinocchio.neutral(model))
    T_zero = {name: oMi for name, oMi in zip(model.names, data.oMi)}

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))
    
    # resolve scale
    x_hat = G.nodes["x"][POS] - G.nodes[model.names[1]][POS]
    y_hat = G.nodes["y"][POS] - G.nodes[model.names[1]][POS]
    z_hat = G.nodes[model.names[1]+'_tilde'][POS] - G.nodes[model.names[1]][POS]

    # resolve rotation and translation
    x = norm(x_hat)
    y = norm(y_hat)
    z = norm(z_hat)
    R = np.vstack((x, -y, z)).T
    B = pinocchio.SE3(R, np.zeros((3,))) #G.nodes[ROOT][POS])

    omega_z = skew(np.array([0,0,1]))

    theta = {}

    prev_rotation = data.oMi[1].rotation
    
    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')
            print((G.nodes[aux_cur][POS] - G.nodes[cur][POS]) / np.linalg.norm(G.nodes[aux_cur][POS] - G.nodes[cur][POS]))
            cur_dir = prev_rotation.transpose()@(G.nodes[aux_cur][POS] - G.nodes[cur][POS]) / np.linalg.norm(G.nodes[aux_cur][POS] - G.nodes[cur][POS])
            theta[pred] = np.arctan2(cur_dir[0],cur_dir[1])
            print(cur,cur_dir)
            prev_rotation = prev_rotation@Rot_z(theta[pred])@T_zero[cur].rotation

    # if the rotation axis of final joint is aligned with ee frame z axis,
    # get angle from EE pose if available
    #if ((T_final is not None) and (la.norm(cross(T_rel.trans, np.asarray([0, 0, 1]))) < tol)):
     #   T_th = (T[cur]).inv().dot(T_final[ee]).as_matrix()
     #   theta[ee] = wraptopi(theta[ee] +  arctan2(T_th[1, 0], T_th[0, 0]))

    return theta


def joint_variables(model,data, G, T_final, axis_length):
    """
    Finds the set of decision variables corresponding to the
    graph realization G.

    :param G: networkx.DiGraph with known vertex positions
    :param T_final: poses of end-effectors in case two final frames aligned along z
    :returns: Dictionary of joint angles
    :rtype: Dict[str, float]
    """
    tol = 1e-10
    ROOT = model.names[1]
    T = {}
    T[ROOT] = data.oMi[1] #G.nodes[model.names[1]][POS]

    pinocchio.forwardKinematics(model,data,pinocchio.neutral(model))
    T_zero = {name: oMi for name, oMi in zip(model.names, data.oMi)}

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))
    
    # resolve scale
    x_hat = G.nodes["x"][POS] - G.nodes[model.names[1]][POS]
    y_hat = G.nodes["y"][POS] - G.nodes[model.names[1]][POS]
    z_hat = G.nodes[model.names[1]+'_tilde'][POS] - G.nodes[model.names[1]][POS]

    # resolve rotation and translation
    x = norm(x_hat)
    y = norm(y_hat)
    z = norm(z_hat)
    R = np.vstack((x, -y, z)).T
    B = pinocchio.SE3(R, np.zeros((3,))) #G.nodes[ROOT][POS])

    omega_z = skew(np.array([0,0,1]))

    theta = {}
    
    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')

            T_prev = T[pred]
            
            T_prev_0 = T_zero[pred] # previous p xf at 0
            T_0 = T_zero[cur] # cur p xf at 0
            T_0_q = pinocchio.SE3.act(T_zero[cur],trans_z) # cur q xf at 0
            T_rel = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0) # relative xf
            ps_0 = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0).translation # relative xf
            qs_0 = pinocchio.SE3.act(pinocchio.SE3.inverse(T_prev_0),T_0_q).translation # rel q xf

            # predicted p and q expressed in previous frame
            p = pinocchio.SE3.act(pinocchio.SE3.inverse(B),G.nodes[cur][POS])
            qnorm = G.nodes[cur][POS] + (
                G.nodes[aux_cur][POS] - G.nodes[cur][POS]
            ) / np.linalg.norm(G.nodes[aux_cur][POS] - G.nodes[cur][POS])
            q = pinocchio.SE3.act(pinocchio.SE3.inverse(B),qnorm)
            #print(T_prev)
            ps = pinocchio.SE3.inverse(T_prev).rotation.dot(p - T_prev.translation)  # in prev. joint frame
            qs = pinocchio.SE3.inverse(T_prev).rotation.dot(q - T_prev.translation)  # in prev. joint frame

            theta[cur] = np.arctan2(-qs_0.dot(omega_z).dot(qs), qs_0.dot(omega_z.dot(omega_z.T)).dot(qs))
            rot_axis_z = pinocchio.SE3(Rot_z(theta[cur]),np.zeros((3,)))
            T[cur] = pinocchio.SE3.act(pinocchio.SE3.act(T_prev,rot_axis_z),T_rel)

    # if the rotation axis of final joint is aligned with ee frame z axis,
    # get angle from EE pose if available
    #if ((T_final is not None) and (la.norm(cross(T_rel.trans, np.asarray([0, 0, 1]))) < tol)):
     #   T_th = (T[cur]).inv().dot(T_final[ee]).as_matrix()
     #   theta[ee] = wraptopi(theta[ee] +  arctan2(T_th[1, 0], T_th[0, 0]))

    return theta

In [35]:
q0 = np.array([30,35,10,-40,90,0,0])*np.pi/180
#q0 = np.array([0,0,0,0,0,0,0])*np.pi/180
#q0 = pinocchio.randomConfiguration(model)
pinocchio.framesForwardKinematics(model,data,q0)
position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
direction = data.oMi[-1].rotation@np.array([0,0,1])

print(position, direction)

G1, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init_ = goal_graph(model, data, G, axis_length, 
                                                                                                 position, direction = direction, anchor = True)



[0.58081515 0.39732814 0.83122999] [0.77377923 0.57562804 0.26442024]


In [92]:
theta = joint_variables(model,data, G1, data.oMi[-1], axis_length)
theta = np.array([theta_ for key, theta_ in theta.items()])
theta,q0

(array([ 0.33637958,  1.20223745,  2.47112297, -0.69964559,  0.48453469,
        -0.64004202]),
 array([ 0.52359878,  0.61086524,  0.17453293, -0.6981317 ,  1.57079633,
         0.        ,  0.        ]))

In [93]:
pinocchio.forwardKinematics(model,data,np.concatenate([theta,np.zeros((1,))]))
#pinocchio.forwardKinematics(model,data,theta)
print(position,direction)
data.oMi[-1]

[0.58081515 0.39732814 0.83122999] [0.77377923 0.57562804 0.26442024]


  R =
-0.419836  0.474036  0.773969
 0.153669 -0.803325  0.575373
 0.894496  0.360498   0.26442
  p = 0.580545 0.397232 0.831253

In [94]:
theta = joint_variables(model,data, G1, data.oMi[-1], axis_length)
theta = np.array([theta_ for key, theta_ in theta.items()])
pinocchio.forwardKinematics(model,data,np.concatenate([theta,np.zeros((1,))]))
print(np.linalg.norm(position-data.oMi[-1].translation))
print(np.linalg.norm(direction-data.oMi[-1].rotation@np.array([0,0,1])))

0.0002871216347693684
0.0003177878825356687


In [81]:
nx.get_node_attributes(G1,POS)

{'lbr_iiwa_joint_1': array([0.    , 0.    , 0.1575]),
 'x': array([2.00000000e-01, 7.21606364e-11, 1.57500000e-01]),
 'y': array([ 3.34216076e-27, -2.00000000e-01,  1.57500000e-01]),
 'lbr_iiwa_joint_1_tilde': array([ 1.27128454e-07, -1.26655382e-07,  3.57500254e-01]),
 'lbr_iiwa_joint_2': array([8.96723306e-05, 3.12153585e-05, 3.59999216e-01]),
 'lbr_iiwa_joint_2_tilde': array([-0.0659426 ,  0.1888161 ,  0.36000014]),
 'lbr_iiwa_joint_3': array([0.18007003, 0.06298349, 0.43392033]),
 'lbr_iiwa_joint_3_tilde': array([0.35619332, 0.12458672, 0.50593198]),
 'lbr_iiwa_joint_4': array([0.36978928, 0.1294167 , 0.51159547]),
 'lbr_iiwa_joint_4_tilde': array([0.36031622, 0.29207782, 0.39561434]),
 'lbr_iiwa_joint_5': array([0.43848888, 0.23146159, 0.64909995]),
 'lbr_iiwa_joint_5_tilde': array([0.51243632, 0.34218077, 0.79834181]),
 'lbr_iiwa_joint_6': array([0.51805903, 0.35073149, 0.80998334]),
 'lbr_iiwa_joint_6_tilde': array([0.61288249, 0.19011093, 0.88216055]),
 'lbr_iiwa_joint_7': arra

In [78]:
W = np.diag(np.concatenate([np.ones(n_anchor),np.ones(n)]))
#Y_D10 = simple_IK(3,C,D,A_,n, n_anchor,max_iter=1000,W=W,Y_init=None)
#Y_D5 = simple_IK(5,C,D,A_anchor,n, n_anchor,max_iter=1000,W=W,Y_init=Y_init_) #[A_,Y_init])
Y_D4 = simple_IK(4,C,D,A_anchor,n, n_anchor,max_iter=2000,W=W,Y_init=None,use_rand =False)
Y_D3 = simple_IK(3,C,D,A_anchor,n, n_anchor,max_iter=2000,W=W,Y_init=Y_D4,use_rand =False)

In [80]:
Y_star = Y_D3
e1 = np.array([1,0,0])
e2 = np.array([0,1,0])
e3 = np.array([0,0,1])

if len(Y_star) == 2:
    X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D,data)
else:
    X = direction_to_position(Y_star,C,D,data)

print(X)
print(np.linalg.norm(X[-2,:]@R.transpose()@R_.transpose()@Rz_flip-position))
print(np.linalg.norm((X[-1,:]-X[-2,:])/axis_length-direction))


for idx,node in enumerate(G1.nodes()):
    G1.nodes[node][POS] = X[idx,:]


[[ 0.00000000e+00  0.00000000e+00  1.57500000e-01]
 [ 2.00000000e-01  7.21606364e-11  1.57500000e-01]
 [ 3.34216076e-27 -2.00000000e-01  1.57500000e-01]
 [ 1.27128454e-07 -1.26655382e-07  3.57500254e-01]
 [ 8.96723306e-05  3.12153585e-05  3.59999216e-01]
 [-6.59425956e-02  1.88816097e-01  3.60000136e-01]
 [ 1.80070026e-01  6.29834910e-02  4.33920332e-01]
 [ 3.56193324e-01  1.24586715e-01  5.05931976e-01]
 [ 3.69789284e-01  1.29416701e-01  5.11595465e-01]
 [ 3.60316218e-01  2.92077818e-01  3.95614336e-01]
 [ 4.38488876e-01  2.31461590e-01  6.49099954e-01]
 [ 5.12436317e-01  3.42180770e-01  7.98341812e-01]
 [ 5.18059031e-01  3.50731487e-01  8.09983342e-01]
 [ 6.12882489e-01  1.90110931e-01  8.82160548e-01]
 [ 5.80815568e-01  3.97327707e-01  8.31230002e-01]
 [ 7.35571464e-01  5.12453295e-01  8.84114032e-01]]
6.03696368749306e-07
2.922775498037573e-07


In [117]:
count_success = 0
err_pos = []
err_or = []
scale = 1

axis_length = 0.4*scale
urdf_filename="GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
# 'GraphIK/graphik/robots/urdfs/panda_arm.urdf'

model    = pinocchio.buildModelFromUrdf(urdf_filename)
data     = model.createData()

G = create_base_graph(model,data,axis_length)

for i in tqdm.tqdm(range(10)):
    q0 = pinocchio.randomConfiguration(model)
    pinocchio.framesForwardKinematics(model,data,q0)
    position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
    direction = None#data.oMi[-1].rotation@np.array([0,0,1])

    #print(position, direction)

    G1, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init_ = goal_graph(model, data, G, axis_length, 
                                                                                                 position, direction = direction, anchor = True)
    
    W = np.diag(np.concatenate([np.ones(n_anchor),np.ones(n)]))
    Y_D4 = simple_IK(4,C,D,A_anchor,n, n_anchor,max_iter=2000,W=W,Y_init=None,use_rand =False)
    Y_D3 = simple_IK(3,C,D,A_anchor,n, n_anchor,max_iter=2000,W=W,Y_init=Y_D4,use_rand =False)

    
    Y_star = Y_D3

    if len(Y_star) == 2:
        #print(ANCHOR)
        X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D,data)
    else:
        X = direction_to_position(Y_star,C,D,data)
    #print((X[-2,:]-X[-1,:])/np.linalg.norm((X[-2,:]-X[-1,:])),direction)
        
    for idx,node in enumerate(G1.nodes()):
        G1 .nodes[node][POS] = X[idx,:]

    theta = joint_variables(model,data, G1, data.oMi[-1], axis_length)
    theta = np.array([theta_ for key, theta_ in theta.items()])
    pinocchio.forwardKinematics(model,data,np.concatenate([theta,np.zeros((1,))]))
    err_pos.append(np.linalg.norm(position-data.oMi[-1].translation))
    #err_or.append(np.linalg.norm(direction-data.oMi[-1].rotation@np.array([0,0,1])))
    #err_or.append(np.linalg.norm((X[-1,:]-X[-2,:])/np.linalg.norm((X[-2,:]-X[-1,:]))-direction))
    #err_pos.append(np.linalg.norm(X[-2,:]-position))
    #print(X[3,:])


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:16<00:00,  1.68s/it]


In [118]:
print(max(err_pos))#,max(err_or))
#sum(np.array(err_or)<10**-5)


9.502164231653188
